In [1]:
!pip install requests_html

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 83 kB 2.5 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 102 kB 3.5 MB/s            
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=89b6059a36d6a4cf9c6bb21f46fb83169c5004e9260c5c6eb1ac5f42f2e47b68
  Stored in directory: /home/praharshb/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=cecb650e612334a56f51f92f16c9fb806ccc2638ad491e415fd40d50d61cca72
  Stored in directory: /home/praharshb/.cache/pip/wheels/ae/e7/76/7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=c867812b0aa9878b80ecb6d8679d20f3c1a

In [2]:
from requests_html import HTMLSession
import pandas as pd
import requests

In [3]:
# Parse particular news for tickers' mentions,takes in an url of an article to pass and an instance of a parsing session from page parser
def url_parse(parse_url, parse_session):
  parse_request = parse_session.get(parse_url)
  content = parse_request.html.find('section.release-body')

  try:
    for item in content:
      parse_ticker = item.find('a.ticket-symbol', first=True).text
  except AttributeError:
    parse_ticker = None
  try:
    return parse_ticker
  except UnboundLocalError:
    return None # Return non if no tickers found

In [4]:
# Function to parse a parse a particular page for all the news to later parse them for tickers. Takes 2 parameters: a number of pages and initial dataset of already saved news.
def page_parse(x, page_session, data=[]):
    page_url = f'https://www.prnewswire.com/news-releases/english-releases/?page={x}&pagesize=100'
    page_request = page_session.get(page_url)
    content = page_request.html.find('div.row.arabiclistingcards')
    for item in content:
        date = item.find('h3', first=True).text.split('ET')[-2]
        title = item.find('h3', first=True).text.split('ET')[-1]
        article_url = 'https://www.prnewswire.com' + item.find('a.newsreleaseconsolidatelink', first=True).attrs['href']
        ticker = url_parse(article_url, page_session)
        try:
          dic = {
              'Date': pd.to_datetime(date),
              'Title': title,
              'Ticker': ticker
          }
        except Exception:
          pass
        data.append(dic)
    return data

In [5]:
# Main scanner, takes a number of pages to parse with 10 being a default. 50 pages cover a week of news and 100 pages cover 2 weeks of data.
def run_scanner(pages=10):
  session = HTMLSession()
  data = []

  for x in range(1, pages+1):
      page_parse(x, session, data)

  df = pd.DataFrame(data)
  df.dropna(subset=['Ticker'], inplace=True)
  df.set_index('Date', inplace=True)
  return df

In [6]:
df = run_scanner(10)

In [7]:
df

,Title,Ticker
Date,,
2021-11-07 13:45:00,"ROSEN, A LONGSTANDING LAW FIRM, Encourages Hö...",HMLP
2021-11-07 12:00:00,FirstEnergy Announces Transformative $3.4 Bil...,FE
2021-11-07 07:05:00,Phase 2 study results of Ionis' novel antisen...,IONS
2021-11-07 02:35:00,Elbit Systems Schedules Third Quarter 2021 Re...,ESLT
2021-11-06 15:06:00,"CEI DEADLINE: Camber Energy, Inc. Investors w...",CEI
...,...,...
2021-11-04 13:51:00,Shareholder Alert: Ademi LLP investigates whe...,PTRS
2021-11-04 13:39:00,Shareholder Alert: Ademi LLP investigates whe...,LEVL
2021-11-04 13:36:00,Shareholder Alert: Ademi LLP investigates whe...,NPTN


In [12]:
df.to_csv('Oct25Nov6.csv') # Save dataframe as a csv for further analysis